# Text Mining for Economics and Finance: Homework 1

## Roger Garriga Calleja, Javier Mas Adell, José Fernando Moreno Gutiérrez

Data: Speech_data_extended.txt found on https://github.com/sekhansen/text-mining-tutorial.

### Exercise 1: Pre-processing the data
Packages used:

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk import PorterStemmer
import sklearn

We will work with data from 1900 onwards to make the data more manageable. However, the same process could be used in the whole dataset. The data we are using corresponds to the speeches made by the different presidents of United States. Each row is a speech and it contains three columns: The president, the speech and the year.

In [ ]:
#Import the dataset.
data = pd.read_table("speech_data_extend.txt", encoding="utf-8")
data = data.loc[data['year']>=1900]
data = data.reset_index()

We need to pre-process the data in order to analyse it. There are four steps:
- Tokenization (separate single words/tokens by spaces).
- Removal of non-alphabetical characters.
- Removal of stopwords (words that do not help diferenciate documents such as 'a', 'the', 'for', etc.).
- Stemming (remove suffixes).

In [ ]:
#Tokenization
prep_data = data.apply(lambda row: nltk.word_tokenize(row['speech'].lower()), axis=1)

#Removal of stop words and non-alphanumeric characters
stop_w=set(stopwords.words('english'))
for i in range(len(prep_data)):
    prep_data[i] = [re.sub('[^\\w]','',w) for w in prep_data[i] if (re.sub('[^\\w]','',w) not in stop_w) and 
                    re.sub('[^\\w]','',w)!=''] 

#Stemming
stemmer = PorterStemmer() #Create a stemmer object
for i in range(len(prep_data)):
    prep_data[i] = [stemmer.stem(elem) for elem in prep_data[i]]

Next step is to create an tf-idf score for each word in the corpus and eliminate those that 